In [1]:
import pandas as pd
import os
import random
import spacy
import json
import re
from spacy.util import minibatch
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
f = open("speech_testing.txt", 'rb')
data = f.readlines()
data_store = pd.DataFrame()
data = [str(each).replace("\'", "").replace("\\n", "")[1:] for each in data][1:]

In [3]:
coms = []
scores = []
for each in range(len(data)):
    elems = data[each].split(";")
    coms.append(elems[0])
    scores.append(1 if float(elems[1]) > 0.5 else 0)
data_store["comment"] = coms
#data_store["comment"].to_csv("trash.csv")
phonemized_ds = pd.read_csv("trash.csv")
#data_store["phonemized"] = phonemized_ds["comment"]
data_store["score"] = scores
phonemized_ds

,Unnamed: 0,comment
0,0,You should know womens sports are a joke
1,1,You look like Sloth with deeper Downs syndrome
2,2,You look like Russian and speak like Indian. B...
3,3,"Women deserve to be abused, I guess."
4,4,Women are made for making babies and cooking d...
...,...,...
993,993,From the midnight sun where the hot springs blow
994,994,Dont say Im not your type
995,995,And therefore never send to know for whom the ...
996,996,And I cant stand another day


In [3]:
final_data = pd.read_csv("bro_please_work.csv")
final_data = final_data.drop(['Unnamed: 0'], axis='columns')
final_data

,comment,phonemized,score
0,You should know womens sports are a joke,juː ʃʊd noʊ wɪmɪnz spoːɹts ɑːɹ ɐ dʒoʊk,1
1,You look like Sloth with deeper Downs syndrome,juː lʊk laɪk slɑːθ wɪð diːpɚ daʊnz sɪndɹoʊm,1
2,You look like Russian and speak like Indian. B...,juː lʊk laɪk ɹʌʃən ænd spiːk laɪk ɪndiən boʊθ ...,1
3,"Women deserve to be abused, I guess.",wɪmɪn dɪzɜːv təbi ɐbjuːzd aɪ ɡɛs,1
4,Women are made for making babies and cooking d...,wɪmɪn ɑːɹ meɪd fɔːɹ meɪkɪŋ beɪbiz ænd kʊkɪŋ dɪ...,1
...,...,...,...
994,And therefore never send to know for whom the ...,ænd ðɛɹfoːɹ nɛvɚ sɛnd tə noʊ fɔːɹ huːm ðə bɛl ...,0
995,And I cant stand another day,ænd aɪ kænt stænd ɐnʌðɚ deɪ,0
996,"All values, unless otherwise stated, are in US...",ɔːl væljuːz ʌnlɛs ʌðɚwaɪz steɪɾᵻd ɑːɹ ɪn juːɛs...,0
997,negha you a little shit samurai,nɛɡhə juː ɐ lɪɾəl ʃɪt sæmjʊɹɹaɪ,1


In [45]:
nlp = spacy.blank('en')
#nlp = en_core_web_md.load()
textcat = nlp.create_pipe('textcat', config = {"executive_classes":True, "architexture":"bow"})
nlp.add_pipe(textcat)

textcat.add_label("no_hate")
textcat.add_label("hate")

train_text = final_data['phonemized'].values
train_labels = [{'cats': {'no_hate' : label == 0, 'hate' : label == 1}} for label in final_data['score']]
train_data = list(zip(train_text, train_labels))
#random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

for epoch in range(150):
    losses = {}
    random.shuffle(train_data)
    batches = minibatch(train_data, size = 10)
    for batch in batches:
        text, labels = zip(*batch)
        nlp.update(text, labels, sgd = optimizer, losses = losses)
    print(epoch+1, end = ' ')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 

In [ ]:
test_phrase = input('Check hate: ')
doc = [nlp.tokenizer(test_phrase)]
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(doc)
predicted_label = scores.argmax(axis=1)
[textcat.labels[label] for label in predicted_label]

In [29]:
misspelled_n_word_and_one_normal = ['fɑːk juː naɪə ', 'fʌk juː nɛɡhə ', 'fʌk juː nɪɡə ', 'juː sænd nɪɡɚ ', 'juː lɪɾəl naɪɑːɹ ', 'fɑːk juː ɡeɪ æs ', 'haɪ fɹɛnd wʌts ʌp ', 'fɑːk juː ']
test_phrase = input('Check hate: ')
doc = [nlp.tokenizer(test_phrase)]
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(doc)
predicted_label = scores.argmax(axis=1)
[textcat.labels[label] for label in predicted_label]

Check hate:  haɪ fɹɛnd wʌts ʌp


['no_hate']

In [46]:
nlp2 = spacy.blank('en')
textcat_new = nlp2.create_pipe('textcat', config = {"executive_classes":True, "architexture":"bow"})
nlp2.add_pipe(textcat_new)

textcat_new.add_label("no_hate")
textcat_new.add_label("hate")

train_text = final_data['comment'].values
train_labels = [{'cats': {'no_hate' : label == 0, 'hate' : label == 1}} for label in final_data['score']]
train_data = list(zip(train_text, train_labels))
#random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp2.begin_training()

for epoch in range(160):
    losses = {}
    random.shuffle(train_data)
    batches = minibatch(train_data, size = 10)
    for batch in batches:
        text, labels = zip(*batch)
        nlp2.update(text, labels, sgd = optimizer, losses = losses)
    print(epoch+1, end = ' ')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 

In [56]:
test_phrase = input('Check hate: ')
doc = [nlp2.tokenizer(test_phrase)]
textcat = nlp2.get_pipe('textcat')
scores, _ = textcat.predict(doc)
predicted_label = scores.argmax(axis=1)
scores, [textcat.labels[label] for label in predicted_label]

Check hate:  you little nigger


(array([[0.11976375, 0.9333883 ]], dtype=float32), ['hate'])

In [7]:
f = open('chat.json', 'r')
data = f.read().replace('\n', '')
cleaned_data = json.loads(data)
messages = {}
temp = {'Author':[], 'Message':[]}
test_set = pd.DataFrame(temp)
pattern = "reacted *.* to your message"
for each in cleaned_data['messages']:
    if 'content' in each.keys():
        sender = each['sender_name']
        msg = each['content']
        if sender in messages and 'an att' not in msg and 'liked a message' not in msg and not re.search(pattern, msg):
            messages[each['sender_name']].append(msg)
            test_set.loc[len(test_set.index)] = [sender, msg]
        elif 'an att' not in msg and 'liked a message' not in msg and not re.search(pattern, msg):
            messages[sender] = [msg]
            test_set.loc[len(test_set.index)] = [sender, msg]

In [5]:
test_set

,Author,Message
0,Kaustubh Joshi,$
1,Puneet Bajaj,Murica
2,Kaustubh Joshi,ð¤
3,Puneet Bajaj,ð«ð¤ ðð¦ ðºð¸
4,Puneet Bajaj,ð¤«
...,...,...
8073,Puneet Bajaj,Lord and 4th roommate
8074,Kartikay Sankhdher,Where
8075,Kaustubh Joshi,Abe tu to bola sab chhodke jars
8076,Puneet Bajaj,Tabhi to summer ka bhi kiraya diya


In [ ]:
res = []
for test_phrase in test_set['Message']:
    doc = [nlp2.tokenizer(test_phrase)]
    textcat_bro = nlp2.get_pipe('textcat')
    scores, _ = textcat_bro.predict(doc)
    predicted_label = scores.argmax(axis=1)
    res.append([textcat_bro.labels[label] for label in predicted_label][0])
    print(test_set['Author'][test_set['Message'] == test_phrase].values[0]+":", test_phrase + " ||| intent:", res[-1], scores)

In [8]:
final_data["comment"][21]

'WE NEED THE NAZI PARTY TO RETURN.....GET ANOTHER HITLER...AND FORM THE SS....AND START AN ETHNIC CLEANSING OF ALL MUSLIMS...YES YES..GATHER THEM UP...PUT THEM IN A CONCENTRATION CAMP...THEN PUT THESE MUSLIM REFUGEES ON A SHIP AND SEND THEM ALL BACK TO THIER OWN COUNTRIES ....AND THEY CAN DO AS THEY WANT THERE...LIVE THIER WAYS AND BY THIER RIDICULOUS LAWS.... THEY DONT ASSIMILATE TO THE NEW WESTERN COUNTRIES..AND TRY TO DICTATE TO US WHAT TO DO AND TO EVEN REMOVE JESUS FROM SCHOOLS AND OTHER PUBLIC AREAS...ARE YOU FUCKING SERIOUS...FUCK OFF YOU DIRTY SMELLING MUSLIMS... STOP DICTATING WHAT WE CAN OR CANT DO IN OUR OWN COUNTRIES...PEOPLE POWER MUST RISE AND SAY ENOUGH IS ENOUGH...TAKE SWEDEN BACK FOR SWEDES...TAKE GREECE BACK FOR GREEKS..TAKE GERMANY BACK..TAKE POLAND BACK..TAKE DENMARK BACK...SAVE OUR COUNTRIES....AND TAKE EUROPE BACK. INTRODUCE AN...SS...STYLE POLICE...AND CLEAN UP THESE STREETS AND GHETTOS....FULL OF DIRTY MUSLIMS...ITS THE GOVERNMENTS FAULT FOR ALLOWING THS SHIT TO